# Aquisição e Análise de Medições

A classe `MDMS` utiliza as configurações definidas no arquivo `config.toml` para inicializar o motor de aquisição para preparar o ambiente para geração das medições sintéticas via OpenDSS.

O sistema realiza a geração de dados sintéticos para:
- Potência ativa (P)  
- Potência reativa (Q)  
- Magnitudes de tensão (|V|)





In [0]:
# Widgets
dbutils.widgets.text("sub", "TAQ", "Subestação")
dbutils.widgets.text("feeder", "TAQ03", "Alimentador")
dbutils.widgets.text("uni_tr_mt", "34102241", "Rede BT")
dbutils.widgets.text("random_state", "42", "Semente Aleatória")
sub = dbutils.widgets.get("sub")
feeder = dbutils.widgets.get("feeder")
uni_tr_mt = dbutils.widgets.get("uni_tr_mt")
random_state = dbutils.widgets.get("random_state")
if random_state == "":
    random_state = None
else:
    random_state = int(random_state)

In [0]:
import os
import sys
current_path = os.getcwd()
parent_path = os.path.dirname(current_path)
sys.path.append(parent_path)

In [0]:
# MDMS engine
from Utils.MDMS import MDMS, config
mdms = MDMS(sub=sub, feeder=feeder, uni_tr_mt=uni_tr_mt, random_state=random_state)
meas = mdms.acquire() # Parâmetro customers

In [0]:
display(mdms.irregs.drop(columns=["theft_kw", "measured_kw"]))

In [0]:
fig = mdms.plot_ntl()

In [0]:
fig = mdms.plot_topology(
    irreg_buses = mdms.irregs["bus_id"].tolist()
)

# Treinamento do Modelo Multilayer Perceptron (MLP)

Treinamento de um modelo MLP voltado à previsão das magnitudes de tensão em um sistema elétrico de baixa tensão. 



In [0]:
dbutils.widgets.text("run_id", "", "Run ID (MLFlow)")
dbutils.widgets.text("use_mlflow", "True", "Habilitar MLFlow")
run_id = dbutils.widgets.get("run_id")
use_mlflow = True if dbutils.widgets.get("use_mlflow").lower()=="true" else False
if run_id != "" and use_mlflow:
    import mlflow.pytorch
    model_uri = f"runs:/{run_id}/model"
    model = mlflow.pytorch.load_model(model_uri)
else:
    import numpy as np
    from Models.MultilayerPerceptron import MLP
    model=MLP(
        input_size=mdms.inputs.shape[-1],
        hidden_layers=list(np.asarray(config.mlp.hidden_layers)*mdms.inputs.shape[-1]),
        output_size=mdms.outputs.shape[-1],
        activation_function=config.mlp.activation_function
    )

In [0]:
from Models.MultilayerPerceptron import Training
engine = Training(
    model, 
    inputs = mdms.training.get("inputs"), 
    outputs = mdms.training.get("outputs"), 
    random_state=random_state
)

In [0]:
model = engine.train(
    log_params={
        "sub": sub,
        "feeder": feeder,
        "uni_tr_mt": uni_tr_mt,
        "random_state": random_state
    },
    use_mlflow = use_mlflow
)

In [0]:
import matplotlib.pyplot as plt
errors = engine.model.errors[:, 5].detach().numpy()*2.3
err = errors.flatten()
sigma, mu = np.std(err), np.mean(err)
x = np.linspace(mu - 4 * sigma, mu + 4 * sigma, 1000)
pdf = (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma) ** 2)
fig = plt.figure(figsize=(4.8, 3.2))
# plt.axvspan(mu - 4 * sigma, mu + 4 * sigma, color='lightblue', alpha=0.5)
plt.plot(x, pdf, color='blue', lw=2.0, alpha=0.8)
plt.hist(err, bins=50, color='gray', density=True, alpha=1)
plt.xlabel('Resíduos (V)')
plt.ylabel('Densidade de probabilidade')
plt.xlim([-1, 1])
plt.ylim([0, 3])
plt.grid(alpha=0.2)
plt.tight_layout()
plt.savefig("SVR.svg")

In [0]:
engine.fig_boxplot

# Identificação da PNT

In [0]:
from Models.MultilayerPerceptron import NTLDetection
detection = NTLDetection(
    model = model,
    inputs = mdms.inference.get("inputs"),
    outputs = mdms.inference.get("outputs"),
)


In [0]:
deltaW = detection.execute(
    rn_threshold=6,
    r2_threshold=0.90,
)
display(deltaW.iloc[:5])

In [0]:
display(mdms.irregs.drop(columns=["measured_kw", "theft_kw"]))

In [0]:
import pandas as pd
df = pd.merge(left=mdms.irregs[["bus_id", "num_phases", "kw_by_phase", "pf", "kwh_theft"]], right=deltaW, on="bus_id", how="right").dropna()
df["error"] = np.round(abs((df["kwh_theft"] - df["deltaW"])/df["kwh_theft"]),2)
display(df)

In [0]:
from Utils.PlotFunctions import boxplot
fig = boxplot(np.asarray(detection.residues)/1.50)
fig.savefig("boxplot.svg")

In [0]:
from Utils.PlotFunctions import plot_inputs, plot_outputs
i = -1
outputs_ = detection.outputs_.query("updt==1 and r2<0.5")
inputs_ = detection.inputs_.iloc[outputs_.index]
y_pred = outputs_.iloc[i].outputs
y_true = outputs_.iloc[i].targets
y_optim = outputs_.iloc[i].optimized
x_input = inputs_.iloc[i].inputs
x_optim = inputs_.iloc[i].optimized
columns = detection.inputs.columns
fig1 = plot_inputs(
    columns = columns, 
    x_input = x_input, 
    x_optim = x_optim
)

In [0]:
fig2 = plot_outputs(
    columns = detection.outputs.columns, 
    y_pred = y_pred, 
    y_true = y_true, 
    y_optim = y_optim
)
fig2.savefig("tensoes.svg")